In [0]:
%run /eReport/PkgGeneral/PrcGeneral

In [0]:
import pandas as pd
from datetime import datetime

class PrcRcdProcesaOrigen():
  
  def __init__(self, id_solicitud):
      self.id_solicitud = id_solicitud
      
      #Obteniendo parametros
      parametros = PrcGeneral(id_solicitud = self.id_solicitud).obtenerParametros()
      self.empresa = parametros['empresa']  
      self.fecha_proceso = parametros['fecha_proceso']  
      self.fecha_proceso_str = self.fecha_proceso.strftime('%Y-%m-%d') 
      
      #Fecha de actualizacion
      fecha_actualizacion = datetime.now()
      self.fecha_actualizacion_str = fecha_actualizacion.strftime("%Y-%m-%d %H:%M:%S")
      
      #Conexion
      connection = PrcGeneral().databaseConnect()
      self.connection = connection
      
      cursor = self.connection.cursor()
      self.cursor = cursor
      
  def diasAtrasoInd(self):
      select_x ="""SELECT C.ID, A.CLIENTE_PRODUCTO, B.DIAS_ATRASO, A.IMPORTE, ROW_NUMBER() OVER(PARTITION BY A.CLIENTE_PRODUCTO ORDER BY B.DIAS_ATRASO DESC) NUM
                                  FROM CLIENTE_PRODUCTO_CONCEPTO AS A JOIN CLIENTE_PRODUCTO AS B
                                  ON A.CLIENTE_PRODUCTO = B.ID
                                  JOIN CLIENTE_DEUDOR_PRODUCTO CDP
                                  ON CDP.ID = B.CLIENTE_DEUDOR_PRODUCTO
                                  JOIN CLIENTE AS C
                                  ON C.ID = B.CLIENTE
                                  JOIN PROCESO_PRODUCTO S
                                  ON S.PROCESO = 3 -- Indicador de Atraso
                                  AND S.PRODUCTO = CDP.PRODUCTO
                                  JOIN PROCESO_PRODUCTO_DETALLE PPD
                                  ON PPD.PROCESO_PRODUCTO = S.ID
                                  AND PPD.GRUPO_CONCEPTO_SALDO = 1
                                  AND PPD.CONCEPTO_SALDO = A.CONCEPTO_SALDO
                                  AND PPD.SITUACION_CREDITO = B.SITUACION_CREDITO
                                        WHERE C.FECHA_PROCESO = """"'"+ self.fecha_proceso_str +"""'
                                        """
      sql_select_x = pd.read_sql_query(select_x, self.connection)
      df_select_x = pd.DataFrame(sql_select_x)
      
      #Agregamos saldo acumulado
      #df_select_x['SALDO_ACUMULADO'] = df_select_x.groupby('NUM')['IMPORTE'].sum()
      saldo_ac = df_select_x.groupby('NUM')
      print(saldo_ac)
      saldo_acu = df_select_x.groupby(['NUM'])['IMPORTE'].sum().reset_index()
      df_select_x['SALDO_ACUM'] = np.where(df_select_x['NUM'] < df_select_x['NUM'], df_select_x['IMPORTE'].sum(), merge['credit_type_x'])

      print(df_select_x)
      print(saldo_acu)
      
PrcRcdProcesaOrigen = PrcRcdProcesaOrigen(1)
PrcRcdProcesaOrigen.diasAtrasoInd()

<pandas.core.groupby.generic.DataFrameGroupBy object at 0x7ff219f23190>
 ID CLIENTE_PRODUCTO DIAS_ATRASO IMPORTE NUM
0 137939 113925 16 102534.0 1
1 137939 113925 16 102534.0 2
2 123282 113926 6 956287.0 1
3 123282 113926 6 956287.0 2
4 132322 113927 6 49966.0 1
... ... ... ... ... ...
62077 127309 142401 0 2772153.0 2
62078 115812 142402 0 273283.0 1
62079 115812 142402 0 273283.0 2
62080 121010 142403 0 137799.0 1
62081 121010 142403 0 137799.0 2

[62082 rows x 5 columns]
 NUM IMPORTE
0 1 1.295975e+10
1 2 1.406439e+10
2 3 1.287014e+09
3 4 3.112072e+07
4 5 2.529239e+07
5 6 2.529239e+07
6 7 2.520061e+07
7 8 2.520061e+07
8 9 3.286607e+07
9 10 3.286607e+07
10 11 3.304139e+07
11 12 3.304139e+07
12 13 2.704971e+07
13 14 2.704971e+07
14 15 2.704146e+07
15 16 2.704146e+07
16 17 2.395410e+07
17 18 2.395410e+07
18 19 2.403763e+07
19 20 2.403763e+07
20 21 3.650803e+07
21 22 3.650803e+07
22 23 3.640800e+07
23 24 3.640800e+07
24 25 2.065705e+07
25 26 2.065705e+07
26 27 2.065705e+07
27 28 2.065705e+07
28 29 3.400494e+07
29 30 3.400494e+07
30 31 3.400494e+07
31 32 3.400494e+07